# Python Lambda Calculus

## Booleans

In [57]:
TRUE = lambda x: lambda y: x

In [58]:
FALSE = lambda x: lambda y: y

### Utility Functions

In [59]:
toBool = lambda x: x(True)(False)

In [60]:
toBool(TRUE)

True

In [24]:
toBool(FALSE)

False

### Logical Operators

In [25]:
AND = lambda x: lambda y: x(y)(FALSE)

In [26]:
OR = lambda x: lambda y: x(TRUE)(y)

In [27]:
NOT = lambda x: x(FALSE)(TRUE)

In [28]:
XOR = lambda x:  lambda y: x(NOT(y))(y)

## Natural Numbers

In [29]:
ZERO = lambda f: lambda x: x

In [30]:
ONE = lambda f: lambda x: f(x)

In [31]:
TWO = lambda f: lambda x: f(f(x))

In [32]:
THREE = lambda f: lambda x: f(f(f(x)))

In [33]:
FOUR = lambda f: lambda x: f(f(f(f(x))))

### Number Functions

In [34]:
SUCC = lambda n: lambda f: lambda x: f(n(f)(x))

In [35]:
PRED = lambda n: n(lambda p: lambda z: z(SUCC(p(TRUE)))(p(TRUE)))(lambda z: z(ZERO)(ZERO))(FALSE)

In [36]:
PLUS = lambda k: lambda n: k(SUCC)(n)

In [42]:
MULT = lambda k: lambda n: lambda f: n(k(f))

In [46]:
MULT = lambda k: lambda n: k(PLUS(n))(ZERO)

In [48]:
EXP = lambda k: lambda n: n(k)

### Utility Functions

In [37]:
toInt = lambda x: x(lambda n: n + 1)(0)

In [38]:
toInt(FOUR)

4

In [39]:
toInt(SUCC(FOUR))

5

In [40]:
toInt(PRED(FOUR))

3

In [41]:
toInt(PLUS(FOUR)(THREE))

7

In [47]:
toInt(MULT(FOUR)(THREE))

12

In [49]:
toInt(EXP(TWO)(FOUR))

16

# Unit Testing

In [50]:
import unittest

In [61]:
class Testing(unittest.TestCase):
    def test_booleans(self):
        self.assertEqual(AND(TRUE)(TRUE), TRUE)
        self.assertEqual(OR(FALSE)(FALSE), FALSE)
        self.assertEqual(NOT(FALSE), TRUE)
        self.assertEqual(XOR(TRUE)(TRUE), FALSE)
    def test_naturals(self):
        self.assertEqual(toInt(ZERO), 0)
        self.assertEqual(toInt(ONE), 1)
        self.assertEqual(toInt(TWO), 2)
        self.assertEqual(toInt(THREE), 3)
        self.assertEqual(toInt(FOUR), 4)
        self.assertEqual(toInt(SUCC(FOUR)), 5)
        self.assertEqual(toInt(PLUS(FOUR)(THREE)), 7)

In [62]:
unittest.main(argv=['first-arg-is-ignored'], exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


In [53]:
FOUR(lambda x: x + 1)(0)

4

In [69]:
print('Chutiya 👧🐶')

Chutiya .👧🐶


In [75]:
PLUS(FOUR)(FOUR)(lambda x: x + '👧🐶')('')

'👧🐶👧🐶👧🐶👧🐶👧🐶👧🐶👧🐶👧🐶'

In [92]:
[(i,'🐶') for i in range(0, 20, 2)]

[(0, '🐶'),
 (2, '🐶'),
 (4, '🐶'),
 (6, '🐶'),
 (8, '🐶'),
 (10, '🐶'),
 (12, '🐶'),
 (14, '🐶'),
 (16, '🐶'),
 (18, '🐶')]

In [93]:
list(map((lambda x: x%7==0),[i for i in range(20)]))

[True,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False]